# Dependancies

## Requirements

In [1]:
#!pip install sentence_transformers langchain openai tqdm datasets asyncio scikit-learn cohere tiktoken umap altair

In [2]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union, Optional
from pydantic.config import ConfigDict

import openai
import instructor

from src.bubble import *
from src.models import *
from src.utilities import *


Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Expérience utilisateur de Darty : 1710335239022x546888753774592000


In [3]:
aspects_df = get("Aspect")

In [4]:
aspects_df.head()

,Company,Project,Rating,SubCategory,Associated_feedback,Date,Category,_id,Explanation
0,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350416999x528701117282035500,1710335410193x146397114086602720,2024-02-07 00:00:00+00:00,1710350416439x770681541783434100,1710359842200x821681263546503200,NaN
1,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350410151x820615919729764900,1710335410193x146397114086602720,2024-02-07 00:00:00+00:00,1710350409745x445953751757016450,1710359842221x274758448864749860,Le clavier Magic Keyboard avec Touch ID et pav...
2,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350421272x199685951705496740,1710335410195x175970366374120320,2024-01-31 00:00:00+00:00,1710350419545x536227079121094340,1710359842248x367738921884239740,Installation professionnelle incluant la repri...
3,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350410151x820615919729764900,1710335410202x536776314861663740,2024-01-05 00:00:00+00:00,1710350409745x445953751757016450,1710359842258x625624914090439300,La machine à laver Bosch offre une simplicité ...
4,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710350419870x657684400109240000,1710335410202x536776314861663740,2024-01-05 00:00:00+00:00,1710350419545x536227079121094340,1710359842268x848738395758196600,Livraison impeccable et très soignée.


# Insights extraction

In [5]:
TYPES_LIST = ['Point positif', 'Nouvelle fonctionnalité', 'Point de douleur', 'Bug']

tags_df = get("Tag", constraints=[])
#types_df = get("Type", constraints=[])
categories_df = get("Category")
subcategories_df = get("SubCategory")

In [6]:
company_infos = bubble_client.get(
    "Company",
    bubble_id=COMPANY_ID,
)
project_infos = bubble_client.get(
    "Project",
    bubble_id=PROJECT_ID,
)

feedback_context = {
    "entreprise": company_infos["Name"],
    "context": company_infos['Context'],
    "role": company_infos['Role'],
    "cible": project_infos['Target'],
    "types": '- '+' \n- '.join(TYPES_LIST),
    "tags": '- '+' \n- '.join([row["Name"]+' : '+row["Description"] for _,row in tags_df.iterrows()]),
    #"types": '- '+' \n- '.join([row["Name"]+' : '+row["Description"] for _,row in types_df.iterrows()]),
    #"insight_types": types_descr,
    #"insight_categories": tags_descr,
    #"question": project_infos['Study_question'],
    #"exemple_commentaire": exemple_commentaire,
    #"example_insights": '\n- '.join(list(examples_insights_df['Insights qui devraient en découler'])),
}

feedback_context

{'entreprise': 'Darty',
 'context': 'Fondée en 1957, Darty est une enseigne française spécialisée dans la distribution d\'électroménager, d\'équipements électroniques et de produits culturels. Rachetée par la Fnac en 2016, elle est aujourd\'hui l\'un des leaders européens de la distribution omnicanale.\n\nÉvènements récents:\n\n    2016: Rachat par la Fnac et création du groupe Fnac Darty.\n    2017: Lancement de la marketplace Darty.com.\n    2018: Déploiement du "Contrat de Confiance Fnac Darty" dans tous les magasins.\n    2019: Lancement de l\'offre de services "Darty+."\n    2020: Accélération de la transformation digitale du groupe.\n    2021: Acquisition de Mistergooddeal, spécialiste du e-commerce en produits reconditionnés.\n    2022: Lancement de la Fnac Darty Academy, une plateforme de formation en ligne.\n\nConcurrents:\n\n    Boulanger\n    Conforama\n    Gitem\n    Amazon\n    Cdiscount\n\nEnjeux:\n\n    Darty doit faire face à une concurrence accrue sur le marché de l\'é

In [7]:
ID_CATEG_NONE = categories_df[categories_df["Name"].isna()].iloc[0]["_id"]
SUBCATEG_NONE = subcategories_df[subcategories_df["Name"].isna()]
ID_CATEG_NONE, SUBCATEG_NONE

('1710350430346x397407027406463360',
                              Company Name                           Project  \
 6   1707313014508x102198350946437700  NaN  1710335239022x546888753774592000   
 13  1707313014508x102198350946437700  NaN  1710335239022x546888753774592000   
 20  1707313014508x102198350946437700  NaN  1710335239022x546888753774592000   
 27  1707313014508x102198350946437700  NaN  1710335239022x546888753774592000   
 34  1707313014508x102198350946437700  NaN  1710335239022x546888753774592000   
 41  1707313014508x102198350946437700  NaN  1710335239022x546888753774592000   
 42  1707313014508x102198350946437700  NaN  1710335239022x546888753774592000   
 
                             Category                               _id  
 6   1710350409745x445953751757016450  1710350412594x226453784069726050  
 13  1710350412899x638565534548052100  1710350416027x546052401208678000  
 20  1710350416439x770681541783434100  1710350418849x710992908170040000  
 27  1710350419545x536227

In [8]:
TypeInsight = enum.Enum("Type de l'insight", [(convert_text_to_constants(t), t) for t in TYPES_LIST])
#types_to_id = {convert_text_to_constants(row.Name): row._id for _, row in types_df.iterrows()}

#TypeInsight = enum.Enum("Type de l'insight", [(convert_text_to_constants(row.Name), row.Name) for _, row in types_df.iterrows()])
TagInsight = enum.Enum("Tag de l'insight", [(convert_text_to_constants(row.Name), row.Name) for _, row in tags_df.iterrows()])
tags_to_id = {convert_text_to_constants(row.Name): row._id for _, row in tags_df.iterrows()}
#type_to_id = {convert_text_to_constants(row.Name): row._id for _, row in types_df.iterrows()}
#type_to_id[convert_text_to_constants('Point de douleur')]

In [9]:
list(TypeInsight)

[<Type de l'insight.POINT_POSITIF: 'Point positif'>,
 <Type de l'insight.NOUVELLE_FONCTIONNALITE: 'Nouvelle fonctionnalité'>,
 <Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>,
 <Type de l'insight.BUG: 'Bug'>]

In [117]:
#FeedbackIndex = enum.Enum("Indice du retour associé", [(str(i), i) for i in range(BATCH_SIZE)])

class InsightDetail(BaseModel):
    detail: str = Field(description="Detail important de l'insight.") 
    associated_indexes: List[int] = Field(description="Indices des retours associés.")


class Insight(BaseModel):
    insight: str = Field(description="Insight, c'est a dire infirmation importante que révèle cette étude à l'entreprise, et lui permettera d'améliorer son experience utilisateur, sa stratégie ou son produit.") 
    insight_type: TypeInsight = Field(description="Type de l'insight, parmis "+', '.join(TYPES_LIST)) 
    associated_indexes: List[int] = Field(description="Indices des retours associés.")
    #insight_tags: List[TagInsight] = Field(description="Tags de l'insight. Peut eventuellement être une liste vide.")
    explication: str = Field(description="Explication de l'insights.") 
    #List[InsightDetail] = Field(description="Détails de l'insights. Peut eventuellement être une liste vide.") 
    #consequences: str = Field(description="Conséquences pour l'entrerpise.")
    #consequences: List[str] = Field(description="Conséquences pour l'entrerpise. Peut eventuellement être une liste vide.")
    #recommandations: List[str] = Field(description="Recommandations pour l'entrerpise. Peut eventuellement être une liste vide.")

class ListInsights(BaseModel):
    insights: List[Insight] = Field(description="Liste des insights les plus intéressants qui ont été déduits.")
    def __str__(self):
        return '\n\n'.join([str(x) for x in self.insights])

#ListInsights.model_json_schema() 

In [124]:

with open('Prompts/fr/prompt_regroupement_create_example.txt') as f:
    prompt_regroupement_create_example = PromptTemplate.from_template(f.read())

example = apply_async_analysis([prompt_regroupement_create_example], ListInsights)
example_clustering_json = example[0].json()


In [90]:
print(example_clustering_json)

{"insights":[{"insight":"Difficultés de communication avec le service client","insight_type":"Point de douleur","associated_indexes":[475,1249,1267,475,1249],"details":"Informations contradictoires données par différents interlocuteurs et manque de suivi des demandes."},{"insight":"Dommages causés aux produits des clients les techniciens de Darty.","insight_type":"Point de douleur","associated_indexes":[1267,1372,1695,1965],"details":"Plusieurs machines à laver sont dites avoir été endommé par le technicien supposé venir la réparer."}]}


In [128]:
with open('Prompts/fr/prompt_regroupement.txt') as f:
    prompt_regroupement = PromptTemplate.from_template(f.read())



prompts = []
subcat_ids = []
for subcat_id, df in aspects_df[aspects_df['Explanation'].notna()].groupby('SubCategory'):
    subcat = subcategories_df[subcategories_df['_id'] == subcat_id].iloc[0]
    cat = categories_df[categories_df['_id'] == subcat['Category']].iloc[0]

    feedbacks = '\n'.join([str(index)+' : '+content for (index, content) in df['Explanation'].items()])
    
    prompts.append(prompt_regroupement.invoke({"feedbacks": feedbacks, "category": cat["Name"]+" : "+subcat['Name'], "example":example_clustering_json, **feedback_context}).text)
    subcat_ids.append(subcat_id)


In [129]:
len(prompts[-12])

127031

In [109]:
#list_insights = apply_async_analysis([prompts[-12]], ListInsights)


In [110]:
print(prompts[0])

CONTEXTE:
Tu est un consultant qui travaille pour l'entreprise Darty

Tu as mené une enquête auprès des Client de cette entreprise concernant la catégorie "Qualité du produit : Fonctionnalité", et en a extrait plusieurs retours.
Ton objectif est d'aider l'entreprise à améliorer son experience utilisateur, améliorer sa rentabilité et adapter sa stratégie. 
Tu cherches à extraires les insights les plus intéressants pour l'entreprise Darty des retours qui t'ont été fait.
Ces insights devront être distincts, fidèle au commentaires et aussi intéressants que possible.
Adopte un style concis et efficace.

Voici un bref rappel du context de Darty:

"
Fondée en 1957, Darty est une enseigne française spécialisée dans la distribution d'électroménager, d'équipements électroniques et de produits culturels. Rachetée par la Fnac en 2016, elle est aujourd'hui l'un des leaders européens de la distribution omnicanale.

Évènements récents:

    2016: Rachat par la Fnac et création du groupe Fnac Darty.
 

In [112]:

#print(prompts[0])
print("Traitement synchronisé de", len(prompts), "prompts.")
list_insights = apply_async_analysis(prompts, ListInsights)


Traitement synchronisé de 36 prompts.


In [136]:
[len(x.insights) for x in list_insights]

[5,
 6,
 3,
 8,
 7,
 4,
 5,
 5,
 4,
 6,
 10,
 10,
 10,
 1,
 2,
 9,
 10,
 10,
 7,
 3,
 10,
 10,
 10,
 10,
 1,
 4,
 10,
 10,
 1,
 10,
 5,
 7,
 10,
 10,
 6,
 10]

In [135]:
list_insights[0].insights

[Insight(insight="Les produits reconditionnés, tels que les appareils de mise sous vide et les téléphones, sont souvent perçus comme peu pratiques et peu efficaces, soulignant un besoin d'amélioration dans la sélection et la qualité des produits reconditionnés.", insight_type=<Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>, associated_indexes=[930, 7251, 10001], explication='Les clients expriment leur insatisfaction concernant la performance et la fiabilité des produits reconditionnés, ce qui peut affecter la réputation de Darty en tant que fournisseur fiable de produits de qualité.'),
 Insight(insight="Les appareils présentant des défauts d'étanchéité ou de construction, comme les problèmes de givre sur les réfrigérateurs No Frost et les défauts d'étanchéité sur les produits Whirlpool, posent des questions de sécurité et de qualité.", insight_type=<Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>, associated_indexes=[643, 1276], explication="Ces problèmes de qualité et

In [143]:
def send_insights(insight_list, cat_id, subcat_id):

    if len(insight_list.insights)>0:
      res = bubble_client.create("Insight",
        [{
          "Company": COMPANY_ID,
          "Project": PROJECT_ID,
          "Name": insight.insight,
          "Category": cat_id,
          "SubCategory": subcat_id,
          "Type": insight.insight_type._value_,
          "Explanation": insight.explication,
          #"Tags": [tags_to_id[convert_text_to_constants(tag._name_)] for tag in insight.insight_tags],
          "Aspects": list(aspects_df.iloc[insight.associated_indexes]._id)[:200],
          "Feedbacks": list(aspects_df.iloc[insight.associated_indexes].Associated_feedback)[:200],
          "Nb Feedbacks": len(list(aspects_df.iloc[insight.associated_indexes].Associated_feedback)),
          }  for insight in insight_list.insights]                     
        )
      insights_id = [x['id'] for x in res]
    else:
      insights_id = []

    """if len(insights_group.consequences)>0:
      res = bubble_client.create("Consequence",
        [{
          "Company": COMPANY_ID,
          "Project": PROJECT_ID,
          "Description": conseq.detail,
          "Name": conseq.title,
          }  for conseq in insights_group.consequences]                     
        )
      consequences_id = [x['id'] for x in res] 
    else:
      consequences_id = []
    """

    """bubble_id = bubble_client.create("Insights Group", {
      "Company": COMPANY_ID,
      "Project": PROJECT_ID,
      "Name": insight.insights,
      "Category": cat_id,
      "SubCategory": subcat_id,
      "Insights": insights_id,
      "Consequences": consequences_id,
      #"Tags": [tags_to_id[tag._name_] for tag in insight.insight_tags],
      })"""

    

for (list_insight_groups, subcat_id) in tqdm(zip(list_insights, subcat_ids)):
  cat_id = subcategories_df[subcategories_df['_id'] == subcat_id].iloc[0].Category
  #for insight in list_insight_groups.insights:
  send_insights(list_insight_groups, cat_id, subcat_id)

  empty_subcat = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==cat_id].iloc[0]._id
  send_insights(list_insight_groups, cat_id, empty_subcat)

  empty_subcat = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==ID_CATEG_NONE].iloc[0]._id
  send_insights(list_insight_groups, ID_CATEG_NONE, empty_subcat)


0it [00:00, ?it/s]